In [ ]:
from dotenv import load_dotenv
import openai, json

load_dotenv()

client = openai.OpenAI()

messages = [];

In [ ]:
def get_weather(city):
  return f"The weather in 33 degrees celcius."

FUNCTION_MAP = {
  'get_weather': get_weather
}


In [6]:
TOOLS = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "A function to get the weather of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description": "The name of the city to get the weather of"
          }
        },
        "required": ["city"]
      }
    }
  }
]

In [ ]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
  if message.tool_calls:
    messages.append(
      {
        "role": "assistant",
        "content": message.content or "",
        "tool_calls": [
          {
            "id": tool_call.id,
            "type": "function",
            "function": {
              "name": tool_call.function.name,
              "arguments": tool_call.function.arguments
            }
          } 
          for tool_call in message.tool_calls
        ],
      }
    )

    for tool_call in message.tool_calls:
      function_name = tool_call.function.name
      arguments = tool_call.function.arguments

      print(f"Calling finction: {function_name} with arguments: {arguments}")

      try:
        arguments = json.loads(arguments)
      except json.JSONDecodeError:
        arguments = {}

      function_to_run = FUNCTION_MAP.get(function_name)

      # arguments는 dict니, dict로 변환
      # ** 연산자: dictionary를 함수의 keyword arguments로 풀어서 전달
      # 예: arguments = {"city": "Seoul"} 일 때
      #     function_to_run(**arguments)는 function_to_run(city="Seoul")과 동일
      result = function_to_run(**arguments)

      messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": function_name,
        "content": result,
      })
    

In [ ]:
def call_ai():
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=TOOLS,
  )
  process_ai_response(response.choices[0].message)
  print(response)
  message = response.choices[0].message.tool_calls
  messages.append({"role": "assistant", "content": message})
  print(f"AI: {message}")

In [8]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: my name is jsj
ChatCompletion(id='chatcmpl-Clny16sRN3Qjel35HtPpXeFgXDrJS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, jsj! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765509525, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_aa07c96156', usage=CompletionUsage(completion_tokens=16, prompt_tokens=64, total_tokens=80, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
AI: Nice to meet you, jsj! How can I assist you today?
User: what is the wather is seoul
ChatCompletion(id='chatcmpl-ClnyYv7BO5VYRS8zXV5JRFtcqV0ny', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message